# 嘗試用`langchain`讀取各種格式的檔案

## 安裝需要的套件

In [17]:
!pip install -r requirements.txt

In [22]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

## 讀一些pdf之類的東西

In [23]:
urls = [
    "https://fundreport.moneydj.com/FileOpenAPI.aspx?A=SPY&B=4&type=GetETFInfo1", # 英文pdf
    "https://www.yuantafunds.com/fund/download/1066%E5%8F%B0%E7%81%A3%E5%8D%93%E8%B6%8A50-%E5%85%AC%E9%96%8B%E8%AA%AA%E6%98%8E%E6%9B%B8.pdf",  # 中文pdf
    "https://i.imgur.com/QU8C5va.jpg",  # 中文jpg
    "https://www.mtsac.edu/webdesign/accessible-docs/word/example03.docx",  # 英文word
    "https://scholar.harvard.edu/files/torman_personal/files/samplepptx.pptx",  # 英文ppt
    "https://dornsife.usc.edu/assets/sites/298/docs/ir211wk12sample.xls",  # 英文excel
]

用Unstructured讀

In [24]:
loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()
len(documents)

6

切割之後進vectordb

In [25]:
print("Splitting documents")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

print("Creating vectorstore")
db = Chroma.from_documents(texts, embeddings)

Splitting documents
Creating vectorstore


驗證看看有沒有找到東西

In [39]:
def query(q):
    docs = db.similarity_search(q)
    return docs[0].page_content

print("中英文pdf:")
print("-------------------------------")
print(query("S&P 500 ETF sponsor"))
print("-------------------------------")
print(query("0050收益分配"))
print("-------------------------------")
print("")
print("中文jpg")
print("-------------------------------")
print(query("Toyota Camry"))
print("-------------------------------")
print(query("高級轎車"))
print("-------------------------------")
print("")
print("英文word/ppt/excel")
print("-------------------------------")
print(query("Word Documents Template"))
print("-------------------------------")
print(query("Sample PowerPoint File"))
print("-------------------------------")
print(query("SalesOrders Smith Desk"))
print("-------------------------------")


中英文pdf:
-------------------------------
to use these trademarks pursuant to a sublicense from State Street Global Advisors Funds Distributors, LLC. SPDR® S&P 500® ETF Trust is not sponsored, endorsed, sold or promoted by S&P, its affiliates
-------------------------------
第十五條 收益分配 第一項 本基金投資所得之現金股利、利息收 入、已實現盈餘配股之股票股利面額 部分、收益平準金、已實現資本利得 扣除資本損失(包括已實現及未實現 之資本損失)及本基金應負擔之各項 成本費用後，為可分配收益。 基金收益分配以當年度之實際可分 配收益餘額為正數方得分配。本基金 每受益權單位之可分配收益低於會 計年度結束日每受益權單位淨資產 價值百分之____，經理公司不予分
-------------------------------

中文jpg
-------------------------------
mais

is Toyota Camry [el4RavlA EAR > AOS Fa > FSSRASER KART BHA R SON THB

Jy 1~4 fi 1~4
-------------------------------
為平衡型基金。

15 民國 111 年 11 月 29 日募集成立「元大全球投資級債券傘型證券投資信託基 金之元大 0 至 2 年投資級企業債券證券投資信託基金」、「元大全球投資級債 券傘型證券投資信託基金之元大 2 至 10 年投資級企業債券證券投資信託基 金」及「元大全球投資級債券傘型證券投資信託基金之元大 10 年以上投資級 企業債券證券投資信託基金」，均為債券型基金。
-------------------------------

英文word/ppt/excel
-------------------------------
Word Documents Template

Main heading: 

Use the Heading 1 style 

## 結論
1. 內建Unstructured套件針對中英文pdf效果都很好
2. 內建Unstructured套件對word/ppt/excel支援也很好
3. 對中文圖像辨識OCR支援不佳。需要再研究使用其他第三方套件。